# Ruleta Francesa

In [86]:
from pyquil import Program, get_qc
from pyquil.gates import H, MEASURE, X, Z
import random

<div style="text-align: center;">
    <img src="./img/ruleta_francesa.jpg" alt="imagen ruleta francesa" style="width: 30%;">
</div>

En la ruleta francesa, los números están asociados a colores específicos:

- El número 0 es de color **verde**.
- Los números del 1 al 36 alternan entre **rojo** y **negro** según su paridad:
    - Los números **impares** son **rojos**.
    - Los números **pares** son **negros**.

Este comportamiento se implementa en el código mediante un diccionario llamado `numeros_colores`, donde cada número se asigna a su color correspondiente.

In [87]:
numeros_colores = {}
numeros_colores[0] = "verde"

for i in range(1, 37):
    if i % 2 == 0:
        numeros_colores[i] = "negro"
    else:
        numeros_colores[i] = "rojo"

## Parte 1 - Implementacion ruleta francesa cuántica

### Croupier


El **Croupier** es el encargado de girar la ruleta cuántica y generar números aleatorios del 0 al 36. Para ello, utiliza 6 qubits que coloca en superposición cuántica mediante compuertas Hadamard, lo que permite que cada qubit esté simultáneamente en estado 0 y 1. Al medir estos qubits, obtiene un número binario que convierte a decimal. Si el número resultante es mayor o igual a 37, simplemente vuelve a girar la ruleta hasta obtener un número válido.

Además de generar números, el croupier administra sus propias monedas (comienza con 20), determina el color de cada número según las reglas de la ruleta francesa (0 es verde, los pares son negros y los impares rojos), y actualiza su saldo ganando monedas cuando los jugadores pierden y perdiendo monedas cuando los jugadores ganan.

In [88]:
class Croupier:
    """Representa al croupier que maneja la ruleta cuántica"""
    
    def __init__(self, monedas_iniciales=20):
        self.monedas = monedas_iniciales
        self.num_qubits = 6  # 2^6 < 37 números
        
    def girar_ruleta(self):
        """Genera un número aleatorio usando qubits"""
        numero = 37
        
        while numero >= 37:
            qc = get_qc(f'{self.num_qubits}q-qvm')
            p = Program()
            ro = p.declare('ro', 'BIT', self.num_qubits)
            
            for i in range(self.num_qubits):
                p += H(i)
            
            for i in range(self.num_qubits):
                p += MEASURE(i, ro[i])
            
            p.wrap_in_numshots_loop(1)
            resultado_obj = qc.run(p)
            resultado = resultado_obj.readout_data.get('ro')
            
            # binario (salida programa) -> decimal
            numero = 0
            for i, bit in enumerate(resultado[0]):
                numero += int(bit) * (2 ** i)
        
        return numero
    
    def obtener_color(self, numero):
        return numeros_colores[numero]
    
    def actualizar_monedas(self, cambio):
        self.monedas += cambio


### Clase Jugador


El **Jugador** es quien realiza las apuestas utilizando computación cuántica para tomar decisiones aleatorias. Primero utiliza 2 qubits en superposición para elegir entre 4 tipos de apuesta posibles:

- **Número específico**: Apostar a cualquier número del 0 al 36
- **Paridad**: Apostar a par o impar
- **Rango**: Apostar a manque (1-18) o passe (19-36)
- **Color**: Apostar a rojo o negro

Una vez decidido el tipo de apuesta mediante medición cuántica, genera la apuesta concreta usando más qubits según sea necesario.

Cada jugador comienza con 10 monedas y gestiona su propio saldo durante el juego. Después de cada ronda, verifica si su apuesta coincide con el resultado de la ruleta aplicando las reglas del juego (por ejemplo, el 0 no cuenta como par ni impar, ni como rojo ni negro). Si gana, recibe una moneda del croupier; si pierde, entrega una moneda. Todo el proceso de toma de decisiones es completamente cuántico e independiente de los demás jugadores.

In [89]:
class Jugador:
    """Representa a un jugador que realiza apuestas cuánticas"""
    
    def __init__(self, nombre, monedas_iniciales=10):
        self.nombre = nombre
        self.monedas = monedas_iniciales
        
    def realizar_apuesta(self):
        """Genera una apuesta aleatoria usando qubits"""
        qc = get_qc('2q-qvm')
        
        p = Program()
        ro = p.declare('ro', 'BIT', 2)
        
        p += H(0)
        p += H(1)
        
        p += MEASURE(0, ro[0])
        p += MEASURE(1, ro[1])
        
        p.wrap_in_numshots_loop(1)
        resultado_obj = qc.run(p)
        resultado = resultado_obj.readout_data.get('ro')
        
        tipo_apuesta = int(resultado[0][0]) * 2 + int(resultado[0][1])
        
        # según tipo apuesta selecionada, ejecutar una u otra
        if tipo_apuesta == 0:
            return self._apostar_numero()
        elif tipo_apuesta == 1:
            return self._apostar_paridad()
        elif tipo_apuesta == 2:
            return self._apostar_rango()
        else:
            return self._apostar_color()
    
    def _apostar_numero(self):
        """Apuesta a un número específico"""
        qc = get_qc('6q-qvm')
        p = Program()
        ro = p.declare('ro', 'BIT', 6)
        
        for i in range(6):
            p += H(i)
        
        for i in range(6):
            p += MEASURE(i, ro[i])
        
        p.wrap_in_numshots_loop(1)
        resultado_obj = qc.run(p)
        resultado = resultado_obj.readout_data.get('ro')
        
        numero = 0
        for i, bit in enumerate(resultado[0]):
            numero += int(bit) * (2 ** i)
        
        if numero >= 37:
            return self._apostar_numero()
        
        return {"tipo": "numero", "valor": numero}
    
    def _apostar_paridad(self):
        """Apuesta a par o impar"""
        qc = get_qc('1q-qvm')
        p = Program()
        ro = p.declare('ro', 'BIT', 1)
        
        p += H(0)
        p += MEASURE(0, ro[0])
        
        p.wrap_in_numshots_loop(1)
        resultado_obj = qc.run(p)
        resultado = resultado_obj.readout_data.get('ro')
        
        paridad = "par" if int(resultado[0][0]) == 0 else "impar"
        return {"tipo": "paridad", "valor": paridad}
    
    def _apostar_rango(self):
        """Apuesta rango (1-18) o (19-36)"""
        qc = get_qc('1q-qvm')
        p = Program()
        ro = p.declare('ro', 'BIT', 1)
        
        p += H(0)
        p += MEASURE(0, ro[0])
        
        p.wrap_in_numshots_loop(1)
        resultado_obj = qc.run(p)
        resultado = resultado_obj.readout_data.get('ro')
        
        if int(resultado[0][0]) == 0:
            rango = "manque"
        else:
            rango = "passe"
        
        return {"tipo": "rango", "valor": rango}
    
    def _apostar_color(self):
        """Apuesta a rojo o negro"""
        qc = get_qc('1q-qvm')
        p = Program()
        ro = p.declare('ro', 'BIT', 1)
        
        p += H(0)
        p += MEASURE(0, ro[0])
        
        p.wrap_in_numshots_loop(1)
        resultado_obj = qc.run(p)
        resultado = resultado_obj.readout_data.get('ro')
        
        if int(resultado[0][0]) == 0:
            color = "rojo"
        else:
            color = "negro"
        
        return {"tipo": "color", "valor": color}
    
    def verificar_apuesta(self, apuesta, numero, color):
        """Verifica si la apuesta ganó"""
        
        if apuesta["tipo"] == "numero":
            return apuesta["valor"] == numero
        
        elif apuesta["tipo"] == "paridad":
            if numero == 0:
                return False
            
            es_par = numero % 2 == 0
            
            if apuesta["valor"] == "par" and es_par:
                return True
            elif apuesta["valor"] == "impar" and not es_par:
                return True
            else:
                return False
        
        elif apuesta["tipo"] == "rango":
            if numero == 0:
                return False
            
            if apuesta["valor"] == "manque":
                if 1 <= numero <= 18:
                    return True
                else:
                    return False
            else:
                if 19 <= numero <= 36:
                    return True
                else:
                    return False
        
        elif apuesta["tipo"] == "color":
            if apuesta["valor"] == color and numero != 0:
                return True
            else:
                return False
        
        return False
    
    def actualizar_monedas(self, cambio):
        """Actualiza las monedas del jugador"""
        self.monedas += cambio

### Clase JuegoRuleta

La clase **JuegoRuleta** es la coordinadora que gestiona la interacción entre el croupier y los dos jugadores. Al inicializarse, crea las tres instancias (un croupier con 20 monedas y dos jugadores con 10 monedas cada uno) y define el número de rondas a jugar. En cada ronda, primero el croupier gira la ruleta para obtener un número, luego ambos jugadores realizan sus apuestas de forma independiente, y finalmente se verifica quién ganó o perdió actualizando las monedas correspondientes: si un jugador gana, recibe 1 moneda del croupier; si pierde, entrega 1 moneda al croupier.

El juego continúa durante todas las rondas establecidas o hasta que algún jugador se quede sin monedas. Al finalizar, la clase muestra un resumen con el estado final de las monedas de cada participante y determina quién es el ganador comparando los saldos de ambos jugadores. Todo el flujo del juego es automático y está impulsado por decisiones cuánticas tanto del croupier como de los jugadores.

In [90]:
class JuegoRuleta:
    """Coordina el juego de ruleta entre croupier y jugadores"""
    
    def __init__(self, num_rondas=10):
        self.croupier = Croupier()
        self.jugador1 = Jugador("Jugador 1")
        self.jugador2 = Jugador("Jugador 2")
        self.num_rondas = num_rondas
    
    def jugar_ronda(self, ronda):
        """Ejecuta una ronda del juego"""
        print(f"\n{'='*60}")
        print(f"RONDA {ronda}")
        print(f"{'='*60}")
        
        # El croupier gira la ruleta
        numero = self.croupier.girar_ruleta()
        color = self.croupier.obtener_color(numero)
        
        print(f"\nEl croupier gira la ruleta...")
        
        # Los jugadores realizan sus apuestas
        apuesta1 = self.jugador1.realizar_apuesta()
        apuesta2 = self.jugador2.realizar_apuesta()
        
        print(f"\n{self.jugador1.nombre} apuesta: {apuesta1['tipo']} - {apuesta1['valor']}")
        print(f"{self.jugador2.nombre} apuesta: {apuesta2['tipo']} - {apuesta2['valor']}")
        
        print(f"\nResultado: {numero} ({color})")
        
        # Verificar apuestas
        gano1 = self.jugador1.verificar_apuesta(apuesta1, numero, color)
        gano2 = self.jugador2.verificar_apuesta(apuesta2, numero, color)
        
        # Actualizar monedas
        if gano1:
            self.jugador1.actualizar_monedas(1)
            self.croupier.actualizar_monedas(-1)
            print(f"{self.jugador1.nombre} GANA!")
        else:
            self.jugador1.actualizar_monedas(-1)
            self.croupier.actualizar_monedas(1)
            print(f"{self.jugador1.nombre} pierde")
        
        if gano2:
            self.jugador2.actualizar_monedas(1)
            self.croupier.actualizar_monedas(-1)
            print(f"{self.jugador2.nombre} GANA!")
        else:
            self.jugador2.actualizar_monedas(-1)
            self.croupier.actualizar_monedas(1)
            print(f"{self.jugador2.nombre} pierde")
        
        # Mostrar estado de monedas
        print(f"\nEstado de monedas:")
        print(f"   {self.jugador1.nombre}: {self.jugador1.monedas} monedas")
        print(f"   {self.jugador2.nombre}: {self.jugador2.monedas} monedas")
        print(f"   Croupier: {self.croupier.monedas} monedas")
    
    def jugar(self):
        """Ejecuta todas las rondas del juego"""
        print("RULETA FRANCESA - VERSIÓN CUÁNTICA 🎰")
        print(f"\nNúmero de rondas: {self.num_rondas}")
        print(f"\nMonedas iniciales:")
        print(f"  {self.jugador1.nombre}: {self.jugador1.monedas}")
        print(f"  {self.jugador2.nombre}: {self.jugador2.monedas}")
        print(f"  Croupier: {self.croupier.monedas}")
        
        for ronda in range(1, self.num_rondas + 1):
            self.jugar_ronda(ronda)
            
            # Verificar si algún jugador se queda sin monedas
            if self.jugador1.monedas <= 0 or self.jugador2.monedas <= 0:
                print("\nUn jugador se ha quedado sin monedas. Fin del juego.")
                break
        
        # Resumen final
        print("RESUMEN FINAL")
        print(f"{self.jugador1.nombre}: {self.jugador1.monedas} monedas")
        print(f"{self.jugador2.nombre}: {self.jugador2.monedas} monedas")
        print(f"Croupier: {self.croupier.monedas} monedas")
        
        # Determinar ganador
        if self.jugador1.monedas > self.jugador2.monedas:
            print(f"\n🏆 ¡{self.jugador1.nombre} es el GANADOR!")
        elif self.jugador2.monedas > self.jugador1.monedas:
            print(f"\n🏆 ¡{self.jugador2.nombre} es el GANADOR!")
        else:
            print("\n🤝 ¡EMPATE!")


In [91]:
print(" EJECUTANDO PARTE 1: RULETA FRANCESA SIN TRAMPAS")

juego_parte1 = JuegoRuleta(num_rondas=10)
juego_parte1.jugar()

 EJECUTANDO PARTE 1: RULETA FRANCESA SIN TRAMPAS
RULETA FRANCESA - VERSIÓN CUÁNTICA 🎰

Número de rondas: 10

Monedas iniciales:
  Jugador 1: 10
  Jugador 2: 10
  Croupier: 20

RONDA 1

El croupier gira la ruleta...

Jugador 1 apuesta: color - negro
Jugador 2 apuesta: rango - passe

Resultado: 2 (negro)
Jugador 1 GANA!
Jugador 2 pierde

Estado de monedas:
   Jugador 1: 11 monedas
   Jugador 2: 9 monedas
   Croupier: 20 monedas

RONDA 2

El croupier gira la ruleta...

Jugador 1 apuesta: rango - passe
Jugador 2 apuesta: rango - manque

Resultado: 32 (negro)
Jugador 1 GANA!
Jugador 2 pierde

Estado de monedas:
   Jugador 1: 12 monedas
   Jugador 2: 8 monedas
   Croupier: 20 monedas

RONDA 3

El croupier gira la ruleta...

Jugador 1 apuesta: color - rojo
Jugador 2 apuesta: rango - passe

Resultado: 16 (negro)
Jugador 1 pierde
Jugador 2 pierde

Estado de monedas:
   Jugador 1: 11 monedas
   Jugador 2: 7 monedas
   Croupier: 22 monedas

RONDA 4

El croupier gira la ruleta...

Jugador 1 apuest

## Ruleta francesa trucada

### CroupierTramposo

La clase **CroupierTramposo** hereda de la clase Croupier pero añade la capacidad de hacer trampas mediante manipulación cuántica. A diferencia del primer croupier, este guarda el estado cuántico completo después de la medición inicial, lo que le permite modificarlo posteriormente. El proceso fuciona de la siguiente manera:
1. primero genera un número aleatorio y guarda su representación binaria.
2. Espía la apuesta de un jugador seleccionado aleatoriamente y verifica si ese jugador ganaría con el resultado actual.
3. Si el jugador va a ganar, el croupier hace trampa eligiendo un qubit aleatorio y aplicándole una compuerta X (NOT) para cambiar su estado, lo que modifica el número final de la ruleta.

In [92]:
class CroupierTramposo(Croupier):

    def girar_ruleta_inicial(self):
        """
        Genera un número aleatorio inicial y guarda el estado cuántico.
        """
        numero = 37
        resultado = None
        
        while numero >= 37:
            qc = get_qc(f'{self.num_qubits}q-qvm')
            p = Program()
            ro = p.declare('ro', 'BIT', self.num_qubits)
            
            for i in range(self.num_qubits):
                p += H(i)
            
            for i in range(self.num_qubits):
                p += MEASURE(i, ro[i])
            
            p.wrap_in_numshots_loop(1)
            resultado_obj = qc.run(p)
            resultado = resultado_obj.readout_data.get('ro')
            
            numero = 0
            for i, bit in enumerate(resultado[0]):
                numero += int(bit) * (2 ** i)
        
        return numero, resultado[0]
    
    def hacer_trampa(self, resultado_inicial, apuesta_jugador, numero_inicial):
        """
        Cambio de un qubit si el jugador está en disposición de ganar.
        """
        # Verificar si el jugador ganaría
        color_inicial = self.obtener_color(numero_inicial)
        gana_jugador = self._verificar_victoria_jugador(apuesta_jugador, numero_inicial, color_inicial)
        
        if gana_jugador:
            print(f"   Croupier hace trampa")
            
            numero = 37  # Inicializar con valor inválido
            
            while numero >= 37:
                # Cambiar un qubit aleatorio
                qc = get_qc(f'{self.num_qubits}q-qvm')
                p = Program()
                ro = p.declare('ro', 'BIT', self.num_qubits)
                
                # Aplicar X a los qubits que eran 1
                for i in range(self.num_qubits):
                    if int(resultado_inicial[i]) == 1:
                        p += X(i)
                
                # Elegir un qubit aleatorio para cambiar estado
                qubit_a_cambiar = random.randint(0, self.num_qubits - 1)
                print(f"   🔄 Cambiando el qubit {qubit_a_cambiar}...")
                
                # Aplicar X (NOT) al qubit seleccionado para cambiar su estado
                p += X(qubit_a_cambiar)
                
                # Medir el nuevo estado
                for i in range(self.num_qubits):
                    p += MEASURE(i, ro[i])
                
                p.wrap_in_numshots_loop(1)
                resultado_obj = qc.run(p)
                resultado = resultado_obj.readout_data.get('ro')
                
                numero = 0
                for i, bit in enumerate(resultado[0]):
                    numero += int(bit) * (2 ** i)
            
            return numero
        else:
            return numero_inicial
    
    def _verificar_victoria_jugador(self, apuesta, numero, color):
        """
        Verifica si un jugador ganaría con su apuesta.
        """
        if apuesta["tipo"] == "numero":
            return apuesta["valor"] == numero
        
        elif apuesta["tipo"] == "paridad":
            if numero == 0:
                return False
            
            es_par = numero % 2 == 0
            
            if apuesta["valor"] == "par" and es_par:
                return True
            elif apuesta["valor"] == "impar" and not es_par:
                return True
            else:
                return False
        
        elif apuesta["tipo"] == "rango":
            if numero == 0:
                return False
            
            if apuesta["valor"] == "manque":
                if 1 <= numero <= 18:
                    return True
                else:
                    return False
            else:
                if 19 <= numero <= 36:
                    return True
                else:
                    return False
        
        elif apuesta["tipo"] == "color":
            if apuesta["valor"] == color and numero != 0:
                return True
            else:
                return False
        
        return False

### Ruleta tramposa

In [ ]:
class JuegoRuletaTramposo(JuegoRuleta):
    """
    Hereda de JuegoRuleta para permitir trampas.
    """
    
    def __init__(self, num_rondas=10):
        self.croupier = CroupierTramposo()
        self.jugador1 = Jugador("Jugador 1")
        self.jugador2 = Jugador("Jugador 2")
        self.num_rondas = num_rondas
        self.trampas_exitosas = 0
    
    def jugar_ronda(self, ronda):
        """
        Sobrescribe el método del padre para implementar la secuencia con trampas.
        """
        print(f"\n{'='*60}")
        print(f"RONDA {ronda}")
        print(f"{'='*60}")
        
        apuesta1 = self.jugador1.realizar_apuesta()
        apuesta2 = self.jugador2.realizar_apuesta()
        
        print(f"\n{self.jugador1.nombre} apuesta: {apuesta1['tipo']} - {apuesta1['valor']}")
        print(f"{self.jugador2.nombre} apuesta: {apuesta2['tipo']} - {apuesta2['valor']}")
        
        print(f"\n El croupier gira la ruleta...")
        numero_inicial, resultado_bits = self.croupier.girar_ruleta_inicial()
        
        # el croupier elige a qué jugador "espiar"
        jugador_espiado = random.choice([1, 2])
        apuesta_espiada = apuesta1 if jugador_espiado == 1 else apuesta2
        
        print(f"  El croupier ha visto la apuesta del jugador {jugador_espiado}")
        
        # El croupier hace trampa si es necesario
        numero_final = self.croupier.hacer_trampa(resultado_bits, apuesta_espiada, numero_inicial)
        
        if numero_final != numero_inicial:
            self.trampas_exitosas += 1
            print(f"  Número cambió de {numero_inicial} a {numero_final}")
        
        color = self.croupier.obtener_color(numero_final)
        
        print(f"\n Resultado final: {numero_final} ({color})")
        
        gano1 = self.jugador1.verificar_apuesta(apuesta1, numero_final, color)
        gano2 = self.jugador2.verificar_apuesta(apuesta2, numero_final, color)
        
        # Actualizar monedas
        if gano1:
            self.jugador1.actualizar_monedas(1)
            self.croupier.actualizar_monedas(-1)
            print(f" {self.jugador1.nombre} GANA!")
        else:
            self.jugador1.actualizar_monedas(-1)
            self.croupier.actualizar_monedas(1)
            print(f" {self.jugador1.nombre} pierde")
        
        if gano2:
            self.jugador2.actualizar_monedas(1)
            self.croupier.actualizar_monedas(-1)
            print(f" {self.jugador2.nombre} GANA!")
        else:
            self.jugador2.actualizar_monedas(-1)
            self.croupier.actualizar_monedas(1)
            print(f" {self.jugador2.nombre} pierde")
        
        print(f"\n Estado de monedas:")
        print(f"   {self.jugador1.nombre}: {self.jugador1.monedas} monedas")
        print(f"   {self.jugador2.nombre}: {self.jugador2.monedas} monedas")
        print(f"   Croupier: {self.croupier.monedas} monedas")
    
    def jugar(self):
        """
        Sobrescribe el método del padre para añadir estadísticas de trampas.
        """
        print(" RULETA FRANCESA - TRAMPOSA")
        print(f"\nNúmero de rondas: {self.num_rondas}")
        print(f"\nMonedas iniciales:")
        print(f"  {self.jugador1.nombre}: {self.jugador1.monedas}")
        print(f"  {self.jugador2.nombre}: {self.jugador2.monedas}")
        print(f"  Croupier: {self.croupier.monedas}")
        
        for ronda in range(1, self.num_rondas + 1):
            self.jugar_ronda(ronda)
            
            # Verificar si algún jugador se queda sin monedas
            if self.jugador1.monedas <= 0 or self.jugador2.monedas <= 0:
                print("\n  Un jugador se ha quedado sin monedas. Fin del juego.")
                break
        
        # Resumen final con estadísticas de trampas
        print("RESUMEN FINAL")
        print(f"{self.jugador1.nombre}: {self.jugador1.monedas} monedas")
        print(f"{self.jugador2.nombre}: {self.jugador2.monedas} monedas")
        print(f"Croupier: {self.croupier.monedas} monedas")
        print(f"\n Trampas exitosas del croupier: {self.trampas_exitosas}")
        
        # Determinar ganador
        if self.jugador1.monedas > self.jugador2.monedas:
            print(f"\n🏆 ¡{self.jugador1.nombre} es el GANADOR!")
        elif self.jugador2.monedas > self.jugador1.monedas:
            print(f"\n🏆 ¡{self.jugador2.nombre} es el GANADOR!")
        else:
            print("\n🤝 ¡EMPATE!")

### Ejecución

In [94]:
juego_parte2 = JuegoRuletaTramposo(num_rondas=10)
juego_parte2.jugar()

 RULETA FRANCESA - TRAMPOSA

Número de rondas: 10

Monedas iniciales:
  Jugador 1: 10
  Jugador 2: 10
  Croupier: 20

RONDA 1

Jugador 1 apuesta: numero - 7
Jugador 2 apuesta: color - rojo

 El croupier gira la ruleta...
  El croupier ha visto la apuesta del jugador 2
   Croupier hace trampa
   🔄 Cambiando el qubit 0...
  Número cambió de 27 a 26

 Resultado final: 26 (negro)
❌ Jugador 1 pierde
❌ Jugador 2 pierde

💰 Estado de monedas:
   Jugador 1: 9 monedas
   Jugador 2: 9 monedas
   Croupier: 22 monedas

RONDA 2

Jugador 1 apuesta: numero - 2
Jugador 2 apuesta: numero - 32

 El croupier gira la ruleta...

Jugador 1 apuesta: numero - 2
Jugador 2 apuesta: numero - 32

 El croupier gira la ruleta...
  El croupier ha visto la apuesta del jugador 2

 Resultado final: 4 (negro)
❌ Jugador 1 pierde
❌ Jugador 2 pierde

💰 Estado de monedas:
   Jugador 1: 8 monedas
   Jugador 2: 8 monedas
   Croupier: 24 monedas

RONDA 3

Jugador 1 apuesta: color - negro
Jugador 2 apuesta: color - negro

 El cr